# AB Model from spec

## Setup

Package installation

In [ ]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [ ]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
from datetime import datetime
import logging
from collections import Counter

# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [ ]:
import config.model_config as cfg
import config.worker as worker
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

## Model Environment testing

Station config import

In [ ]:
station_config['A-B']['CS_AB_1']
# worker.sum_total_charging_stations(station_config)

In [ ]:
# {'A-B': {'CS_A-B_1': [{'CPID': 'CS_A-B_1_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_1_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_1_3',
#     'Power': '60',
#     'Distance': '40',
#     'Price': '10',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_2': [{'CPID': 'CS_A-B_2_1',
#     'Power': '7',
#     'Distance': '20',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_2_2',
#     'Power': '7',
#     'Distance': '20',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_3': [{'CPID': 'CS_A-B_3_1',
#     'Power': '7',
#     'Distance': '50',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_3_2',
#     'Power': '7',
#     'Distance': '50',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_3_3',
#     'Power': '60',
#     'Distance': '50',
#     'Price': '10',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_4': [{'CPID': 'CS_A-B_4_1',
#     'Power': '7',
#     'Distance': '10',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_4_2',
#     'Power': '7',
#     'Distance': '10',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_4_3',
#     'Power': '60',
#     'Distance': '10',
#     'Price': '10',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_5': [{'CPID': 'CS_A-B_5_1',
#     'Power': '7',
#     'Distance': '30',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_5_2',
#     'Power': '7',
#     'Distance': '30',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_5_3',
#     'Power': '7',
#     'Distance': '30',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}]},
#  'A-C': {'CS_A-C_1': [{'CPID': 'CS_A-C_1_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_1_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-C_2': [{'CPID': 'CS_A-C_2_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_2_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-C_3': [{'CPID': 'CS_A-C_3_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_3_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-C_4': [{'CPID': 'CS_A-C_4_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_4_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}]}}


In [ ]:
# worker.get_routes(station_config)                   #works
worker.find_cpid_for_charging_station(station_config, 'CS_A-C_1')                 # doesnt work, anymore due to new structure

In [ ]:
# worker.count_charging_stations('A-B', station_config)



# worker.get_target_charging_stations('A-B', station_config)                   # Works
# worker.sum_total_charging_stations(station_config)                           # Works   
# worker.get_routes(station_config)                                            # Works
# worker.total_route_length(station_config, 'A-B')                             # Works
# worker.cpids_for_route(station_config, 'A-B') # not working

# worker.get_checkpoint_list(station_config, 'A-C')                           # Works
# worker.get_charging_stations_on_route(station_config, 'A-C')                    # Works
# worker.charge_points_on_route(station_config, 'A-B')                    # not working
# worker.get_route_cps(station_config, 'A-B')                              #not working snymore

# worker.find_cpid_for_charging_station(station_config, 'CS_AB_1_1')                    # not works



In [ ]:
a = worker.get_routes(station_config)
print(len(a))  
print(a)

In [ ]:
# worker.get_checkpoint_list(station_config, 'A-B')
# worker.charge_points_on_route('A-B', station_config)
# worker.get_charge_points_per_station('A-B', station_config)

In [ ]:
station_config = worker.read_charging_data(cfg.STATION_PATH +'stations.csv')

In [ ]:
worker.num_stations_per_route(station_config)

In [ ]:
# station_config   

In [ ]:
# worker.get_stations_for_route('A-B', station_config)
# stations = worker.get_stations_for_route('A-B', station_config)
# print(stations)  # Output: ['CS_A-B_1', 'CS_A-B_2', 'CS_A-B_3', 'CS_A-B_4', 'CS_A-B_5']


From input matrix to model params

Approach 2

In [ ]:
station_config = worker.read_charging_stations(cfg.STATION_PATH +'stations.csv')

In [ ]:
route_counts = Counter([station['Route'] for station in station_config])
print(route_counts)

In [ ]:
points_per_station_per_route = {}
for station in station_config:
    key = f"{station['Route']}_{station['Station']}"
    if key not in points_per_station_per_route:
        points_per_station_per_route[key] = []
    points_per_station_per_route[key].append(station['CPID'])
# print(points_per_station_per_route)
print(points_per_station_per_route.keys())

In [ ]:
routes = list(set([station['Route'] for station in station_config]))
print(routes)

In [ ]:
num_routes = len(routes)
print(num_routes)

## Valid Model Environment

Approach 3

In [ ]:
station_config = worker.read_csv(cfg.STATION_PATH +'stations.csv')

In [ ]:
# station_config = worker.read_charging_data(cfg.STATION_PATH +'stations.csv')
station_config

In [ ]:
# get the number of charging stations per route
stations_per_route = {route: len(station_config[route]) for route in station_config}
print(stations_per_route)

In [ ]:
# get the number of charging points per station per route
points_per_station_per_route = {}
for route in station_config:
    for station in station_config[route]:
        num_points = len(station_config[route][station])
        if route not in points_per_station_per_route:
            points_per_station_per_route[route] = {}
        points_per_station_per_route[route][station] = num_points
print(points_per_station_per_route)

In [ ]:
# get a list of all routes
routes = station_config.keys()
print(routes)

In [ ]:
# get the total number of routes
num_routes = len(routes)
print(num_routes)

In [ ]:
# worker.get_charge_points_per_station_on_route(station_config, 'A-B')
cs = worker.count_charge_points_by_station(station_config, 'A-B')                        #works
# type(cs)
# b = cs.values()
# b = list(b)
# type(b)
cs

In [ ]:
def get_values(d):
    """
    Returns a list containing the associated values for each key in the dictionary.
    """
    return list(d.values())
# my_dict = {'a': 1, 'b': 2, 'c': 3}
# values_list = get_values(my_dict)
values_list = get_values(cs)
print(values_list)  # Output: [1, 2, 3]

# a = worker.count_charge_points_by_station(station_config, 'A-B') 
# # list = get_values(a) 
# # list = get_values(a) 
# print(a)
# type(a)
# print(a.keys())
# b = list(a.keys())
# print(b)



In [ ]:
# worker.get_checkpoint_list(station_config, 'A-C')                           #  not work
a = worker.get_charging_stations_on_route(station_config, 'A-B') 
print(a)

In [ ]:
cs = worker.get_route_from_config('A-C', station_config)  
# print(cs.keys())
print(cs)

In [ ]:
def cumulative_cs_distances(numbers):
    result = []
    for i in range(len(numbers)):
        if i == 0:
            result.append(numbers[i])
        else:
            result.append(numbers[i] + result[i-1])
    return result

def get_values(d):
    """
    Returns a list containing the associated values for each key in the dictionary.
    """
    return list(d.values())


In [ ]:
a = worker.get_route_from_config('A-C', station_config)

b = list(a.keys())
c = cumulative_cs_distances(b)
# print(a)
print(c)

In [ ]:
# worker.get_cumulative_distances('A-B', station_config)
list = ['a', 'b', 'c', 'd', 'e']

In [ ]:
a = worker.remove_list_item_seq(list)
print(a)
b = worker.remove_list_item_seq(list)
print(b)
c = worker.remove_list_item_seq(list)
print(c)
d = worker.remove_list_item_seq(list)
print(d)
e = worker.remove_list_item_seq(list)
print(e)
print(len(list))

In [ ]:
class MyListClass:
    def __init__(self, n, string_list):
        """
        # This function creates an ’n’ number of class attributes named as each string in the list, and initialised as empty lists.
        """
        for s in string_list:
            setattr(self, s, [])
            for i in range(n):
                getattr(self, s).append(None)

                


In [ ]:
my_obj = MyListClass(5, ['foo', 'bar', 'baz'])
print(my_obj.foo)
# [None, None, None, None, None]

my_obj.foo[0] = 42
print(my_obj.foo)
# [42, None, None, None, None]

my_obj.bar.append('hello')
print(my_obj.bar)
# [None, None, None, None, None, 'hello']


In [ ]:
def select_route_as_key(input):
    """
    This function returns one of the keys in input dictionary, up to the integer value of the key
    """
    # calculate the sum of values in the input dictionary
    n = sum(input.values())
    
    # create a dictionary to keep track of the number of times each key is returned
    counter = {key: 0 for key in input}
    
    # select a key and return it as a string
    def helper():
        for key in input:
            if counter[key] < input[key]:
                counter[key] += 1
                return str(key)
        # if all keys have been returned the maximum number of times, raise an exception
        raise Exception('no more route assignments')
    
    # keep track of the number of times the function has been run
    num_runs = 0
    
    # run the function at most n times
    while num_runs < n:
        try:
            key = helper()
            num_runs += 1
            yield key
        except Exception as e:
            yield str(e)
            return


In [ ]:
input_dict = {'key1': 4, 'key2': 1, 'key3': 1}
for key in select_route_as_key(input_dict):
    print(key)

In [ ]:
def get_charging_stations_along_route(station_config, route_name):
    """
    Returns a dictionary of charging stations along the route. 
    The key is the station name and the value is the distance from the start of the route.   
    """
    charging_stations = {}
    route_stations = station_config.get(route_name)
    if route_stations:
        for station_name, station_data in route_stations.items():
            for station in station_data:
                charging_stations[station_name] = int(station['Distance'])
    return charging_stations




In [ ]:
# a = get_charging_stations_along_route(station_config, 'A-B')
# print(a)
b = worker.get_dict_values(get_charging_stations_along_route(station_config, 'A-B'))
print(b)


Model parameters

In [ ]:
# 365 days = 8760 hours, 30 days = 720 hours, 2 days = 48 hours, 7 days = 168 hours
ticks = 48
no_evs = 2

Run Model

In [ ]:
# model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css, no_cps=no_cps)
# for i in range(ticks):
#     model_run.step()

# # last stable
# model_run = model.EVModel(ticks=ticks, no_evs=no_evs, params=params)
# for i in range(ticks):
#     model_run.step()

model_run = model.EVModel(ticks=ticks, no_evs=no_evs, params=station_config)
for i in range(ticks):
    model_run.step()

In [ ]:
# import numpy as np

# def matrix_to_list(matrix):
#     # Convert the matrix to a nested list
#     nested_list = matrix.tolist()

#     # Return the nested list
#     return nested_list
# matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# nested_list = matrix_to_list(matrix)
# print(nested_list)


In [ ]:
# model_run.schedule.agents
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

Export results to CSV file

In [ ]:
# today's date as string
date_str = str(datetime.today())
# export csv
model_run.datacollector.get_model_vars_dataframe().to_csv(cfg.DATA_PATH + 'data_' + date_str[0:10] + '_' + str(no_evs) + '_EV_agent_model_output.csv')

## Data import and preprocessing

Data import and Helper functions

In [ ]:
import analysis.helper as helper
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [ ]:
data = pd.read_csv(cfg.DATA_PATH +'modeldatadata_2023-03-19_500_EV_agent_model_output.csv')

data = data.drop(columns=['Unnamed: 0'])

In [ ]:
# data.info()
data.head()

In [ ]:
a = helper.unpack_and_join(data, 'EVs Charge Level')
print(a)

In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

In [ ]:
# trip decisions [a-b,b-a,a-c,c-a]
# These change at end of day randomly

## Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

## Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

## Scrapbook